# Obtain data of AGN from SIMBAD and filtering Ne II and Ne III Passband from NED data

The code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

Here we use [Astroquery](https://astroquery.readthedocs.io/en/latest/index.html) package to retriev the data.

In [1]:
from datetime import date

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [3]:
today = date.today()

The sample results is saved in a .csv file:

In [4]:
simbad_sample_Sy = result.to_table()
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because when we search the information in [NED](https://ned.ipac.caltech.edu/) using the object id that we obtained from SIMBAD, sometimes it doesn't reconigze it, so we will use coordinates instead

In [5]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

In the next part of the code we are going to filter the samples that have Ne II, Ne III and Ne V data. First we define a variable with the names of the **"Observed Passband"** for Ne II, Ne III and Ne V. Then with the first loop we will get the photometry table for each object sample, specifically the "Observed Passband Colum" of that tables for search in the second loop all the objects that have data in the Ne passbands

In [75]:
Ne_PB=['[Ne II] 12.81 (IRS)','[Ne II] 12.8 Spitzer','[Ne II] 12.8 (IRS)','[Ne V] 14.32 (IRS)',
       '[Ne V] 14.32 Spitzer','[Ne V] 14.32','[Ne V] 14.32 (IRS)','[Ne V] 14.3 Spitzer',
       '[Ne III] 15.56 (IRS)','[NeIII] 15.6 Spitzer','[Ne III] 15.56 (IRS)','[Ne III] (Spitzer)']


for i in range(100):
    print(i)
    result_table = Ned.query_region(cat_smb_sample[i],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample
    print('\n')
    print(result_table) # an astropy.table.Table
    obj_name=result_table["Object Name"] #Preferred object name in NED
    phot_table = Ned.get_table(obj_name, table='photometry') #We get the photometry table from NED for each sample
    #print(phot_table["Observed Passband"])
    len_table = len(phot_table) #Lenght of the photometry data
    #print(len_table)
     
    for j in range(len_table):
        Passband=phot_table["Observed Passband"][j] #We get the name for each Observed Passband
        if (Passband==Ne_PB[0] or Passband==Ne_PB[1] or Passband==Ne_PB[2] or Passband==Ne_PB[3] or 
            Passband==Ne_PB[4] or Passband==Ne_PB[5] or Passband==Ne_PB[6] or Passband==Ne_PB[7] or
            Passband==Ne_PB[8] or Passband==Ne_PB[9] or Passband==Ne_PB[10] or Passband==Ne_PB[11]):
            #print(result_table) # an astropy.table.Table
            print(phot_table["Observed Passband","Photometry Measurement"][j])
           
            
    
        
    

0


No.        Object Name            RA     ... Diameter Points Associations
                               degrees   ...                             
--- ------------------------- ---------- ... --------------- ------------
  1 WISEA J140931.24+051131.6  212.38024 ...               4            0
 Observed Passband  Photometry Measurement
------------------- ----------------------
[Ne II] 12.81 (IRS)               1.33e-21
Observed Passband  Photometry Measurement
------------------ ----------------------
[Ne V] 14.32 (IRS)                     --
 Observed Passband   Photometry Measurement
-------------------- ----------------------
[Ne III] 15.56 (IRS)               2.01e-21
1


No.        Object Name            RA     ... Diameter Points Associations
                               degrees   ...                             
--- ------------------------- ---------- ... --------------- ------------
  1 WISEA J100346.54+122201.2  150.94397 ...               8            0
2


No.      

  1 WISEA J073126.67+452217.3  112.86121 ...               6            0
28


No.        Object Name            RA     ... Diameter Points Associations
                               degrees   ...                             
--- ------------------------- ---------- ... --------------- ------------
  1 WISEA J073433.00+473404.9   113.6376 ...               4            0
29


No. Object Name     RA     ... Redshift Points Diameter Points Associations
                 degrees   ...                                             
--- ----------- ---------- ... --------------- --------------- ------------
  1  MSPM 00581   182.2867 ...               1               0            0
  2    NGC 4135  182.28671 ...              15              14            0
30


No.        Object Name            RA     ... Diameter Points Associations
                               degrees   ...                             
--- ------------------------- ---------- ... --------------- ------------
  1 WISEA J07

RemoteServiceError: The remote service returned the following error message.
ERROR:  No Photometric data points found.

In [67]:
simbad_sample_Sy[33]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
SDSS J085416.75+502631.9,133.56973333333335,50.44226944444444,2006AJ....131.1163S,Sy2,2016ApJ...832...67N,27314.34577581347,0.095668,z,14,Sy2|AGN|G


In [78]:
pht_table = Ned.get_table('WISEA J085416.75+502631.9', table='photometry')
pht_table

No.,Observed Passband,Photometry Measurement,Uncertainty,Units,Frequency,Flux Density,Upper limit of uncertainty,Lower limit of uncertainty,Upper limit of Flux Density,Lower limit of Flux Density,NED Uncertainty,NED Units,Refcode,Significance,Published frequency,Frequency Mode,Coordinates Targeted,Spatial Mode,Qualifiers,Comments
,,,,,Hz,Jy,,,,,,,,,,,,,,
int32,object,float64,object,object,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object
1,0.3-8 keV (Chandra),3.65e-15,,erg/cm^2^/s,1e+18,3.65e-10,--,--,--,--,,Jy,2016ApJS..224...40W,no uncertainty reported,4.15 keV,Broad-band measurement,,Flux integrated from map,Average flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
2,FUV (GALEX) AB,--,>21.1829,mag,1950000000000000.0,--,--,--,--,1.22e-05,<1.22E-05,Jy,2012GASC..C...0000S,3 sigma,1538.6 A,Broad-band measurement,133.57001879110 50.442080871433 (J2000),Flux integrated from map,upper limit inside NUV Kron aperture,From new raw data
3,FUV (GALEX) AB,22.5324,+/-0.536957,mag,1950000000000000.0,3.52e-06,1.74e-06,1.74e-06,--,--,+/-1.74E-06,Jy,2012GASC..C...0000S,uncertainty,1538.6 A,Broad-band measurement,133.57001879110 50.442080871433 (J2000),Flux in fixed aperture,Flux in 7.5 arcsec diameter aperture,From new raw data
4,NUV (GALEX) AB,21.8486,+/-0.279454,mag,1290000000000000.0,6.62e-06,1.7e-06,1.7e-06,--,--,+/-1.70E-06,Jy,2012GASC..C...0000S,uncertainty,2315.7 A,Broad-band measurement,133.57001879110 50.442080871433 (J2000),Flux integrated from map,Kron flux in elliptical aperture,From new raw data
5,NUV (GALEX) AB,22.2206,+/-0.290090,mag,1290000000000000.0,4.7e-06,1.25e-06,1.25e-06,--,--,+/-1.25E-06,Jy,2012GASC..C...0000S,uncertainty,2315.7 A,Broad-band measurement,133.57001879110 50.442080871433 (J2000),Flux in fixed aperture,Flux in 7.5 arcsec diameter aperture,From new raw data
6,u (SDSS PSF) AB,20.797,+/-0.074,asinh mag,836000000000000.0,1.81e-05,1.23e-06,1.23e-06,--,--,+/-1.23E-06,Jy,2004SDSS2.C...0000:,based on count statistics only,3585 A,Broad-band measurement,133.569839 50.442207 (J2000),Modelled datum,SDSS flags: CHILD - object is part of a blended parent object; BAD_RADIAL - some low S/N radial points; PETROFAINT - Petrosian radius measured at very low surface brightness; BINNED1 - detected at >=5 sigma in original imaging frame; BINNED_CENTER - image was binned while centroiding; BRIGHTEST_GALAXY_CHILD - brightest child among one parent's children;,From new raw data
7,u (SDSS Model) AB,19.463,+/-0.064,asinh mag,836000000000000.0,6.18e-05,3.66e-06,3.66e-06,--,--,+/-3.66E-06,Jy,2004SDSS2.C...0000:,based on count statistics only,3585 A,Broad-band measurement,133.569839 50.442207 (J2000),Modelled datum,SDSS flags: CHILD - object is part of a blended parent object; BAD_RADIAL - some low S/N radial points; PETROFAINT - Petrosian radius measured at very low surface brightness; BINNED1 - detected at >=5 sigma in original imaging frame; BINNED_CENTER - image was binned while centroiding; BRIGHTEST_GALAXY_CHILD - brightest child among one parent's children;,From new raw data
8,u (SDSS CModel) AB,19.337,,asinh mag,836000000000000.0,6.69e-05,--,--,--,--,,Jy,2004SDSS2.C...0000:,no uncertainty reported,3585 A,Broad-band measurement,133.569839 50.442207 (J2000),Modelled datum,SDSS flags: CHILD - object is part of a blended parent object; BAD_RADIAL - some low S/N radial points; PETROFAINT - Petrosian radius measured at very low surface brightness; BINNED1 - detected at >=5 sigma in original imaging frame; BINNED_CENTER - image was binned while centroiding; BRIGHTEST_GALAXY_CHILD - brightest child among one parent's children;,From new raw data
